# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset **tanpa label** dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.
   
2. **Ketentuan Dataset**:
   - **Tanpa label**: Dataset tidak boleh memiliki label atau kelas.
   - **Jumlah Baris**: Minimal 1000 baris untuk memastikan dataset cukup besar untuk analisis yang bermakna.
   - **Tipe Data**: Harus mengandung data **kategorikal** dan **numerikal**.
     - *Kategorikal*: Misalnya jenis kelamin, kategori produk.
     - *Numerikal*: Misalnya usia, pendapatan, harga.

3. **Pembatasan**:  
   Dataset yang sudah digunakan dalam latihan clustering (seperti customer segmentation) tidak boleh digunakan.

## Ship Performance Clustering Dataset  
Dataset yang digunakan yaitu Ship Performance Clustering Dataset yang berasal dari sumber [kaggle](https://www.kaggle.com/datasets/jeleeladekunlefijabi/ship-performance-clustering-dataset)  

### Deskripsi Dataset  
Dataset in merupakan kumpulan data sintetis namun realistis yang dirancang untuk mewakili metrik dan atribut operasional utama dari berbagai jenis kapal di Teluk Guinea. Dataset ini dirancang untuk para penggemar analisis data maritim, praktisi pembelajaran mesin, dan profesional yang tertarik untuk mengeksplorasi masalah pengelompokan, prediksi, dan pengoptimalan dalam industri maritim. Dataset ini bertujuan untuk menyediakan platform untuk mengeksplorasi tren kinerja kapal, mengidentifikasi pola, dan memecahkan tantangan maritim dunia nyata melalui pendekatan berbasis data.  

### Fitur Dataset  
Dataset ini terdiri dari 2736 baris dan 24 kolom, dengan fitur-fitur yang dikategorikan ke dalam jenis numerik dan kategorikal. Di bawah ini adalah fitur yang ada:

**Fitur Numerik**
- Speed_Over_Ground_knots: Kecepatan rata-rata kapal di atas air (dalam knot).
- Engine_Power_kW: Output daya mesin (dalam kilowatt).
- Distance_Traveled_nm: Total jarak yang ditempuh oleh kapal (dalam mil laut).
- Operational_Cost_USD: Total biaya operasional per pelayaran (dalam USD).
- Revenue_per_Voyage_USD: Pendapatan yang dihasilkan per pelayaran (dalam USD).
- Efficiency_nm_per_kWh: Efisiensi energi yang dihitung dalam mil laut per kilowatt-jam.

**Fitur Kategorikal**
- Ship_Type: Jenis kapal (misalnya, Kapal Tanker, Kapal Kontainer, Kapal Pengangkut Ikan, Kapal Curah).
- Route_Type: Jenis rute pelayaran (misalnya, Jarak pendek, Jarak jauh, Transoceanic).
- Engine_Type: Jenis mesin (misalnya, Diesel, Bahan Bakar Minyak Berat).
- Maintenance_Status: Kondisi perawatan kapal (misalnya, Cukup, Kritis, Baik).
- Weather_Condition: Kondisi cuaca yang berlaku selama pelayaran (misalnya, Tenang, Sedang, Kasar).

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
#Type your code here
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer  # Mengimpor KElbowVisualizer untuk visualisasi metode Elbow
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

In [ ]:
df = pd.read_csv('Dataset_clustering.csv')
df.head()

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. EDA bertujuan untuk:

1. **Memahami Struktur Data**
   - Tinjau jumlah baris dan kolom dalam dataset.  
   - Tinjau jenis data di setiap kolom (numerikal atau kategorikal).

2. **Menangani Data yang Hilang**  
   - Identifikasi dan analisis data yang hilang (*missing values*). Tentukan langkah-langkah yang diperlukan untuk menangani data yang hilang, seperti pengisian atau penghapusan data tersebut.

3. **Analisis Distribusi dan Korelasi**  
   - Analisis distribusi variabel numerik dengan statistik deskriptif dan visualisasi seperti histogram atau boxplot.  
   - Periksa hubungan antara variabel menggunakan matriks korelasi atau scatter plot.

4. **Visualisasi Data**  
   - Buat visualisasi dasar seperti grafik distribusi dan diagram batang untuk variabel kategorikal.  
   - Gunakan heatmap atau pairplot untuk menganalisis korelasi antar variabel.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
#Type your code here
# Menampilkan informasi tentang dataset, termasuk jumlah baris, kolom, tipe data, dan jumlah nilai non-null
df.info()

In [ ]:
# Kode ini digunakan untuk mengecek apakah ada missing value
df.isna().sum()

In [ ]:
# Kode ini digunakan untuk mengecek apakah ada data duplikasi
print("Jumlah duplikasi pada dataset : ", df.duplicated().sum())

In [ ]:
#  Kode ini digunakan untuk mengecek parameter statistik dari tabel hour
df.describe(include='all')

In [ ]:
# Menyimpan kolom numerik ke dalam variabel column_numeric
column_numeric = df.select_dtypes(include=['number']).columns
column_numeric 

In [ ]:
# Menyimpan kolom kategorik ke dalam variabel column_categorical, kecuali kolom 'Date'
column_categorical = df.select_dtypes(exclude=['number']).columns.drop('Date', errors='ignore')
column_categorical

In [ ]:
# Membuat histogram untuk melihat distribusi data pada kolom numerik
for i in column_numeric:
    plt.figure(figsize=(10,5))
    sns.histplot(df[i], kde=True)
    plt.title(i)
    plt.show()

In [ ]:
# Menampilkan boxplot untuk melihat distribusi data pada kolom numerik
for i in column_numeric:
    plt.figure(figsize=(10,5))
    sns.boxplot(df[i])
    plt.title(i)
    plt.show()

In [ ]:
# Visualiasi korelasi antar kolom numerik
plt.figure(figsize=(12,8))
sns.heatmap(df[column_numeric].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Visualisasi korelasi antar kolom numerik dengan scatter plot
sns.pairplot(df[column_numeric])
plt.suptitle('Scatter Plot Pairwise Correlation of Numerical Columns', fontsize=35, fontweight='bold', y=1)
plt.show()

In [ ]:
# Visualisasi distribusi data pada kolom kategorik dengan bar plot dan kecuali kolom date
for i in column_categorical:
    if i != 'Date':
        plt.figure(figsize=(15,10))
        sns.barplot(x=df[i].value_counts().index, y=df[i].value_counts())
        plt.title(i)
        plt.show()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning. Data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah.

In [ ]:
#Type your code here
# Menghapus kolom date
df = df.drop(columns='Date')
df.head()

In [ ]:
# Menghapus missing value
df = df.dropna()

In [ ]:
# Mengecek apakah masih ada missing value
df.isna().sum()

In [ ]:
df.info()

In [ ]:
# Menampilkan nilai unik dan jumlahnya untuk setiap kolom kategorikal
for col in column_categorical:
    unique_values = df[col].unique()
    print(f"Kolom: {col}")
    print(f"Nilai unik: {unique_values}")
    print(f"Jumlah nilai unik: {len(unique_values)}\n")

# PERCOBAAN

# **6. Pembangunan Model Clustering**

## **a. Pembangunan Model Clustering**

Pada tahap ini, Anda membangun model clustering dengan memilih algoritma yang sesuai untuk mengelompokkan data berdasarkan kesamaan. Berikut adalah **rekomendasi** tahapannya.
1. Pilih algoritma clustering yang sesuai.
2. Latih model dengan data menggunakan algoritma tersebut.

In [ ]:
#Type your code here
# Inisialisasi model KMeans tanpa parameter awal
kmeans = KMeans()


## **b. Evaluasi Model Clustering**

Untuk menentukan jumlah cluster yang optimal dalam model clustering, Anda dapat menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu kita menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

In [ ]:
#Type your code here

## **c. Feature Selection (Opsional)**

Silakan lakukan feature selection jika Anda membutuhkan optimasi model clustering. Jika Anda menerapkan proses ini, silakan lakukan pemodelan dan evaluasi kembali menggunakan kolom-kolom hasil feature selection. Terakhir, bandingkan hasil performa model sebelum dan sesudah menerapkan feature selection.

In [ ]:
#Type your code here

## **d. Visualisasi Hasil Clustering**

Setelah model clustering dilatih dan jumlah cluster optimal ditentukan, langkah selanjutnya adalah menampilkan hasil clustering melalui visualisasi.

Berikut adalah **rekomendasi** tahapannya.
1. Tampilkan hasil clustering dalam bentuk visualisasi, seperti grafik scatter plot atau 2D PCA projection.

In [ ]:
#Type your code here

## **e. Analisis dan Interpretasi Hasil Cluster**

### Interpretasi Target

**Tutorial: Melakukan Inverse Transform pada Data Target Setelah Clustering**

Setelah melakukan clustering dengan model **KMeans**, kita perlu mengembalikan data yang telah diubah (normalisasi, standarisasi, atau label encoding) ke bentuk aslinya. Berikut adalah langkah-langkahnya.

---

**1. Tambahkan Hasil Label Cluster ke DataFrame**
Setelah mendapatkan hasil clustering, kita tambahkan label cluster ke dalam DataFrame yang telah dinormalisasi.

```python
df_normalized['Cluster'] = model_kmeans.labels_
```

Lakukan Inverse Transform pada feature yang sudah dilakukan Labelisasi dan Standararisasi. Berikut code untuk melakukannya:
label_encoder.inverse_transform(X_Selected[['Fitur']])

Lalu masukkan ke dalam kolom dataset asli atau membuat dataframe baru
```python
df_normalized['Fitur'] = label_encoder.inverse_transform(df_normalized[['Fitur']])
```
Masukkan Data yang Sudah Di-Inverse ke dalam Dataset Asli atau Buat DataFrame Baru
```python
df_original['Fitur'] = df_normalized['Fitur']
```

In [ ]:
# Type your code here


### Inverse Data Jika Melakukan Normalisasi/Standardisasi

Inverse Transform untuk Data yang Distandarisasi
Jika data numerik telah dinormalisasi menggunakan StandardScaler atau MinMaxScaler, kita bisa mengembalikannya ke skala asli:
```python
df_normalized[['Fitur_Numerik']] = scaler.inverse_transform(df_normalized[['Fitur_Numerik']])
```

In [ ]:
# Type your code here

Setelah melakukan clustering, langkah selanjutnya adalah menganalisis karakteristik dari masing-masing cluster berdasarkan fitur yang tersedia.

Berikut adalah **rekomendasi** tahapannya.
1. Analisis karakteristik tiap cluster berdasarkan fitur yang tersedia (misalnya, distribusi nilai dalam cluster).
2. Berikan interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis? Apakah ada pola tertentu yang bisa dimanfaatkan?

In [ ]:
# Type your code here

Tulis hasil interpretasinya di sini.
1. Cluster 1:
2. Cluster 2:
3. Cluster 3:

# Contoh interpretasi [TEMPLATE]
# Analisis Karakteristik Cluster dari Model KMeans

Berikut adalah analisis karakteristik untuk setiap cluster yang dihasilkan dari model KMeans.

## Cluster 1:
- **Rata-rata Annual Income (k$):** 48,260  
- **Rata-rata Spending Score (1-100):** 56.48  
- **Analisis:** Cluster ini mencakup pelanggan dengan pendapatan tahunan menengah dan tingkat pengeluaran yang cukup tinggi. Pelanggan dalam cluster ini cenderung memiliki daya beli yang moderat dan mereka lebih cenderung untuk membelanjakan sebagian besar pendapatan mereka.

## Cluster 2:
- **Rata-rata Annual Income (k$):** 86,540  
- **Rata-rata Spending Score (1-100):** 82.13  
- **Analisis:** Cluster ini menunjukkan pelanggan dengan pendapatan tahunan tinggi dan pengeluaran yang sangat tinggi. Pelanggan di cluster ini merupakan kelompok premium dengan daya beli yang kuat dan cenderung mengeluarkan uang dalam jumlah besar untuk produk atau layanan.

## Cluster 3:
- **Rata-rata Annual Income (k$):** 87,000  
- **Rata-rata Spending Score (1-100):** 18.63  
- **Analisis:** Cluster ini terdiri dari pelanggan dengan pendapatan tahunan yang tinggi tetapi pengeluaran yang rendah. Mereka mungkin memiliki kapasitas finansial yang baik namun tidak terlalu aktif dalam berbelanja. Ini bisa menunjukkan bahwa mereka lebih selektif dalam pengeluaran mereka atau mungkin lebih cenderung untuk menyimpan uang.

# **7. Mengeksport Data**

Simpan hasilnya ke dalam file CSV.